CRAWL DATA

In [ ]:
# === CRAWL ===

from Libraries import Crawler

my_config = {
    # --- Path ---
    "DATA_DIR": "Database",
    
    # --- Filter ---
    "MIN_YEAR": 2020,
    "MIN_WORDS": 200,
    "MAX_WORDS": 1000,
    
    # --- Quy trình crawl ---
    "TARGET_ARTICLES_PER_SUBTYPE": 100,
    "MAX_CONCURRENT_WORKERS": 10,
    "VALIDATION_ARTICLES_COUNT": 10,
    
    # --- Chuyên mục muốn crawl ---
    "TYPE_DICT": {
        'thoi-su': ['chinh-tri', 'dan-sinh', 'lao-dong-viec-lam', 'giao-thong', 'mekong', 'quy-hy-vong'],
        'goc-nhin': ['binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-suc-khoe', 'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong', 'van-hoa-doi-song', 'covid-19', 'tac-gia'],
        'the-gioi': ['tu-lieu', 'phan-tich', 'nguoi-viet-5-chau', 'cuoc-song-do-day', 'quan-su'],
        'kinh-doanh': ['net-zero', 'quoc-te', 'doanh-nghiep', 'chung-khoan', 'ebank', 'vi-mo', 'tien-cua-toi', 'hang-hoa'],
        'bat-dong-san': ['chinh-sach', 'thi-truong', 'khong-gian-song', 'tu-van'],
        'khoa-hoc': ['khoa-hoc-trong-nuoc', 'pii-doi-moi-sang-tao', 'tin-tuc', 'phat-minh', 'ung-dung', 'the-gioi-tu-nhien', 'thuong-thuc'],
        'giai-tri': ['gioi-sao', 'sach', 'video', 'phim', 'nhac', 'thoi-trang', 'lam-dep', 'san-khau-my-thuat'],
        'the-thao': ['bong-da', 'du-lieu-bong-da', 'marathon', 'tennis', 'cac-mon-khac', 'hau-truong'],
        'phap-luat': ['ho-so-pha-an', 'tu-van'],
        'giao-duc': ['tin-tuc', 'tuyen-sinh', 'chan-dung', 'du-hoc', 'thao-luan', 'hoc-tieng-anh', 'giao-duc-40'],
        'suc-khoe': ['tin-tuc', 'cac-benh', 'song-khoe', 'vaccine'],
        'doi-song': ['nhip-song', 'to-am', 'bai-hoc-song', 'cooking', 'tieu-dung'],
        'du-lich': ['diem-den', 'am-thuc', 'dau-chan', 'tu-van', 'cam-nang'],
        'so-hoa': ['cong-nghe', 'san-pham', 'blockchain'],
        'xe': ['thi-truong', 'dien-dan'],
        'y-kien': ['thoi-su', 'doi-song']
    }
}

# === RUN === 
crawler = Crawler(config=my_config)
crawled_data = crawler.run()

TRAIN and TEST

In [ ]:
import os
import pandas as pd
from transformers import pipeline
from Libraries import Trainer

training_config = {
    # --- Đường dẫn và tên model ---
    "DATA_JSONL_FILE": "Database/JSON/vnexpress_articles.jsonl",
    "MODEL_CHECKPOINT": "vinai/bartpho-syllable",
    "OUTPUT_MODEL_DIR": "Models/bartpho-summarizer-v1",
    
    # --- Hyperparameters ---
    "MAX_INPUT_LENGTH": 1024,
    "MAX_TARGET_LENGTH": 256,
    "BATCH_SIZE": 4,
    "NUM_TRAIN_EPOCHS": 3,
    "LEARNING_RATE": 3e-5,
    "WEIGHT_DECAY": 0.01,
}

In [ ]:
# === TRAIN ===
summarizer_trainer = Trainer.SummarizationTrainer(config=training_config)
summarizer_trainer.run()

In [ ]:
# === TEST ===
fine_tuned_model_path = training_config["OUTPUT_MODEL_DIR"] 
summarizer_pipeline = pipeline("summarization", model=fine_tuned_model_path)

# Lấy một bài báo từ dữ liệu của bạn để tóm tắt thử
df = pd.read_json(training_config["DATA_JSONL_FILE"], lines=True)
sample_text = df.iloc[50]["content"] # Số 50

print("--- VĂN BẢN GỐC ---")
print(sample_text)
print("\n" + "="*50 + "\n")

print("--- BẢN TÓM TẮT TỪ MODEL ---")
summary = summarizer_pipeline(sample_text, max_length=256, min_length=50, do_sample=False)
print(summary[0]['summary_text'])